In [ ]:
# Get the chromosome positions in PRS313 from ./positions/locations.txt
PRS313_positions = pd.read_excel('../../Data/PRS313.xlsx')

# Rename the SNPa column to have a "chr" at the beginning
PRS313_positions['SNPa'] = 'chr' + PRS313_positions['SNPa'].astype(str)



In [ ]:
# Filter the data to only include the positions in PRS313 

excluded_data = data.loc[:, ~data.columns.isin(PRS313_positions['SNPa'])]


data = data.loc[:, data.columns.isin(PRS313_positions['SNPa'])]

data


In [ ]:
data.max().max()

In [ ]:
excluded_data

In [ ]:
import numpy as np

def replace_non_index(value, index):
    
    alleles = value.split('|')
    
    processed_alleles = []
    for allele in alleles:
        if int(allele) == 0 or int(allele) != int(index):
            processed_alleles.append("0")
        else:
            processed_alleles.append("1")
    

    if (int(alleles[0]) > 1):
        print("PRS313 SNP: ", index)
        print("Original Data: ",alleles,"Processed Data: ", '|'.join(processed_alleles))

    return '|'.join(processed_alleles)

# Loop through the missing positions and add them to the data with the values in excluded_data
for index, row in missing_positions.iterrows():


    chromosome = row.Chromosome
    position = row.Positionb
    snp = 'chr' + str(chromosome) + '_' + str(position)
    PRS313_alternate_alleles = row["Effect Allele"]

    # Find in excluded data the columns with names that begin with snp
    snp_data = excluded_data.filter(like=snp)

    excluded_data_snp = snp_data.columns[0]

    excluded_data_snp_alternate_alleles = excluded_data_snp.split('_')[3].split(',')
    
    # Find index of PRS313_alternate_alleles in excluded_data_snp_alternate_alleles

    index = excluded_data_snp_alternate_alleles.index(PRS313_alternate_alleles) + 1

    # print("PRS313 SNP: ", PRS313_alternate_alleles)
    # print("Excluded data SNP: ", excluded_data_snp_alternate_alleles)
    # print("Alternate Allele value of PRS313 SNP in 1000 Genomes VCF: ",index)

    snp_data = snp_data.applymap(lambda x: replace_non_index(x, index))
            
    
    # Add snp_data to data
    data = pd.concat([data, snp_data], axis=1)

In [ ]:
# Get the number of rows where a value of greater than 1 appeared
data.loc[(data > 1).any(axis=1)]


In [ ]:
# Get the number of columns where a value of greater than 1 appeared
data.loc[:, (data > 1).any()]